In [5]:
#Import library
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import glob
from nltk.stem import SnowballStemmer

In [17]:
#Reading file
art1 = """Your Stemming 2020 letter letter 100 of the 7th was received night before last. I very
cheerfully send you the twenty dollars, which sum you say is
necessary to save your land from sale. It is singular that you
should have forgotten a judgment against you; and it is more
singular that the plaintiff should have let you forget it so long,
particularly as I suppose you have always had property enough to
satisfy a judgment of that amount. Before you pay it, it would be
well to be sure you have not paid it; or, at least, that you can
not prove you have paid it. Give my love to Mother, and all the
connections."""
data_pos = pd.read_csv("/Volumes/DATA/aclImdb/pos.csv", sep=';', error_bad_lines=False)
data_pos.columns = ["target", "text"]

In [35]:
##Task 2

#Split in sentences

#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#print ('\n-----\n'.join(tokenizer.tokenize(a)))


#Remove stopwords and punctuations
def remove_stop_words(text):
    stopwords_en = stopwords.words("english")
    stopwords_en = stopwords_en + list(string.punctuation)
    cleaned_tokens = []
    for w in nltk.word_tokenize(text):
        w = w.lower()
        if w not in stopwords_en:
            cleaned_tokens.append(w)
    cleaned_text = " ".join(cleaned_tokens)
    return cleaned_text

def split_sentence(article_text):
    sentences = nltk.sent_tokenize(article_text)
    return sentences

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

def make_bigram(corpus):
    tokens = tokenize(corpus)
    bigram = nltk.ngrams(tokens, n = 2)
    return bigram

def generate_bow(document):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(document)
    print(vectorizer.get_feature_names())
    print(X.toarray())

def tfidf(corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([art1])
    tfidf = pd.DataFrame(X.toarray().transpose(), index=vectorizer.get_feature_names())
    return tfidf

### Task 3 Create pipeline for implementing Task 1, part 1 & 2

In [31]:
#Import library
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.pipeline import FeatureUnion

In [32]:
class CleanText(BaseEstimator, TransformerMixin):

    def _init_(self):
        return
    
    #def _init_(self):
        #return

    def fit( self, X, y = None ):
        return self

    def clean (self, x):
        cleaned_text = remove_stop_words(x)
        tokens = tokenize(cleaned_text)
        return tokens

    def transform( self, X, y = None ):
        return self.clean(X)

In [15]:
from sklearn.pipeline import Pipeline
pipel_task1 = Pipeline([('clean', CleanText())])
pipel_task1.transform(art1)

['stem',
 'letter',
 'letter',
 '100',
 '7th',
 'receiv',
 'night',
 'befor',
 'last',
 'veri',
 'cheer',
 'send',
 'twenti',
 'dollar',
 'sum',
 'say',
 'necessari',
 'save',
 'land',
 'sale',
 'singular',
 'forgotten',
 'judgment',
 'singular',
 'plaintiff',
 'let',
 'forget',
 'long',
 'particular',
 'suppos',
 'alway',
 'properti',
 'enough',
 'satisfi',
 'judgment',
 'amount',
 'befor',
 'pay',
 'would',
 'well',
 'sure',
 'paid',
 'least',
 'prove',
 'paid',
 'give',
 'love',
 'mother',
 'connect']

## Task B

In [29]:
data = pd.read_csv("/Volumes/DATA/aclImdb/movie_reviews1.csv", sep=';', error_bad_lines=False)
data.columns = ["target", "reviews"]
data['target'] = np.where(data['target']=='positive',1,0)
pd.set_option('max_colwidth', 200)
data.head()

,target,reviews
0,0,"I came at this film with high expectations. I was aware of Greenaway's work and 'The Tempest' and was interested in an adaptation. I first wanted to switch off after ten minutes, but felt that it ..."
1,0,POSSIBLE SPOILERSThe Spy Who Shagged Me is a muchly overrated and over-hyped sequel. International Man of Mystery came straight out of the blue. It was a lone star that few people had heard of. Bu...
2,1,It has said that The Movies and Baseball both thrived during The Great Depression. It appears that the grim realities of a Nation caught up in the aftermath of this Economic Disaster created a nee...
3,0,Boring and appallingly acted(Summer Pheonix). She sounded more Asian than Jewish. Some of the scenes and costumes looked more mid 20th century than late 19th century. What on earth fine actors lik...
4,1,"I just came back from the Late-night cinema and it was indeed a silent way out as most of the audience pondered though the real-life black & white images of the partition,the freedom movement,etc ..."


In [54]:
from sklearn.model_selection import train_test_split
X = data['reviews']
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [55]:
#Pre-processing pipeline for Part B
class PreProcess(BaseEstimator, TransformerMixin):

    def _init_(self,):
        return

    def fit( self, X, y = None ):
        return self

    def clean (self, x):
        cleaned_text = remove_stop_words(x)
        #Remove number
        cleaned_text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", cleaned_text)
        return cleaned_text

    def transform(self, X, y = None):
        return X.apply(self.clean)

In [57]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler

pipe_clean = Pipeline([('clean', PreProcess()), ('tfidf', TfidfVectorizer())])

pipe_svm = Pipeline([('pre_process', pipe_clean),
                    ('select', SelectKBest(score_func = chi2, k = 1000)),
                     ("std_scaler", StandardScaler(with_mean = False)),
                    ('classify', LinearSVC(max_iter=10000, tol=0.1, dual = False))])
pipe_svm.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('pre_process',
                 Pipeline(memory=None,
                          steps=[('clean', PreProcess()),
                                 ('tfidf',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 1), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  sto...
     

In [59]:
y_pred = pipe_svm.predict(X_test)

In [61]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.8838109992254067

In [70]:
msg = pd.Series('''The rest are either tasteless or rehashed from IMOM.If this were the first movie of the series then I'd probably be easier on it. But the series started on a note of dry wit and then plummeted down to a level of gross out humour''')
pipe_svm.predict(msg)

array([1])